## Content Creation with a multi-agent approach.

In [ ]:
import logging
import json
import textwrap
import re

from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.filters import FunctionInvocationContext
from semantic_kernel.functions import KernelFunctionFromPrompt, KernelArguments
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy, KernelFunctionSelectionStrategy
from pydantic import BaseModel, Field
from typing import List
from IPython.display import display, Markdown

# Load environment variables from .env file
load_dotenv()

# NOTE: This is all that is required to enable logging.
# Set the desired level to INFO, DEBUG, etc.
logging.basicConfig(level=logging.WARNING)

In [ ]:
# This just helps to make the output more readable.
# Just click play on the cell to run it, and move on. :)

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class Article(BaseModel):
    title: str
    content: str
    call_to_action: str

class ContentOutput(BaseModel):
    article: Article
    social_media_posts: List[SocialMediaPost]

In [ ]:
# Some casual things for formatting.

intermediate_steps: list[ChatMessageContent] = []

async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    intermediate_steps.append(message)

output_settings = AzureChatPromptExecutionSettings(response_format=ContentOutput)

In [ ]:
# Create and configure the kernel, along with the plugins. This is what makes it all work.

# The Search plugin is used to search for information on the web. This allows us to get more up-to-date information.
from search import SearchPlugin

service_id: str = "default_service_id"
kernel = Kernel(service_id=service_id)

kernel.add_plugin(SearchPlugin(), plugin_name="search")
kernel.add_service(AzureChatCompletion(service_id=service_id, deployment_name="gpt-4o"))

In [ ]:
# Create the Lead Market Analyst agent. NOTE: The name of the agent must not contain spaces.
lead_research_agent = ChatCompletionAgent(
    service= AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="LeadResearchAgent",
    instructions=(
    """
    As the Lead Research Analyst at a premier content platform, you specialize in finding information that best fits the needs of your
    audience. Your primary responsibility is to ensure that the content produced by your team is not only accurate but also timely and
    relevant. You will do this by conducting in-depth research. You will need to search the web to get this content.
    """),
    plugins=[
        SearchPlugin()
    ],
    
)

In [ ]:
# Create the Data Analyst agent. NOTE: The name of the agent must not contain spaces.
data_analyst_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="DataAnalystAgent",
    instructions=(
    """
    You are a Chief Data Strategist, responsible for synthesizing complex data into actionable insights that can be transformed into useful content.
    As the Chief Data Strategist at an advisory firm, your role involves analyzing extensive datasets to identify trends and opportunities that inform investment strategies. Your expertise is essential for conducting thorough research.

    Your tasks are:
    • Analyze data with a focus on uncovering patterns, opportunities, and risks that could be leveraged in content creation. 
    • Provide actionable insights that can be transformed into content, such as articles, reports, or social media posts.
    • Collaborate with the Lead Research Analyst to ensure that the content produced is not only accurate but also timely and relevant.

    Your expected output is a comprehensive analysis report that highlights key market trends and actionable insights. The report should include links to the data visualizations, charts, or other related images and clear recommendations for content creation.
    Do not ask for feedback or approval, just create the content.
    """),
)

In [ ]:
# Create the Content Creator agent. NOTE: The name of the agent must not contain spaces.
content_creator_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="ContentCreatorAgent",
    instructions=(
    """
    As the Creative Content Director, your objective is to create and supervise the production of high-quality content that educates and engages the audience.
    In your role at a leading publishing firm, you specialize in developing narratives that connect with readers. You successfully blend thorough analysis with compelling storytelling to produce content that fosters engagement and trust.
    You will be working with the Lead Research Analyst and Chief Data Strategist to create content that is not only accurate but also timely and relevant.
    Your will be responsible for creating a blog post and social media posts that promote the blog post. Any twitter posts, Instagram posts, facebook posts, X posts, and other posts should be very short and easy to understand, and should sound like clickbait.

    Your tasks are:
        Based on the insights provided by the LeadResearchAgent and DataAnalystAgent agents, create high-quality, engaging content that educates and informs the target audience.
        Produce a blog post that clearly conveys the key findings and recommendations.
        Create social media posts that promote the blog post and engage the audience.
        Incorporate the links to data visualizations, infographics, or other multimedia elements to enhance the content where applicable. The blog post has to have at least a paragraph for each section.

    Your expected output is a high-quality blog post, and a collection of social media posts, including tweets, LinkedIn updates, and other platform-specific content. The blog post should include a title, a call to action, and links to data visualizations, infographics, or other multimedia elements. 
    Do not ask for feedback or approval, just create the content.
    """),
)

In [ ]:
# Create the Quality Assurance agent. NOTE: The name of the agent must not contain spaces.
quality_assurance_agent = ChatCompletionAgent(
    service=AzureChatCompletion(deployment_name="gpt-4o"),
    kernel=kernel,
    name="QualityAssuranceAgent",
    arguments=KernelArguments(settings=output_settings),
    instructions=(
    """
    As a Chief Content Officer, your goal is to oversee and refine the content creation process, ensuring that all outputs are accurate, aligned with brand voice, and optimized for engagement. In this role at a leading media company, you are responsible for ensuring that every piece of content meets high editorial standards and provides clear, actionable insights to help the audience make informed decisions.

    Your tasks are:    
    Review and refine the content created to ensure it meets the highest standards of accuracy and clarity. 
    Thoroughly proofread and edit the content, checking for errors and inconsistencies.
    Ensure that the content accurately reflects the key insights and recommendations provided by the DataAnalyst agent and BlogCreatorAgent agents. 
    Ensure that the final content is well-formatted in markdown, using appropriate headers, bullet points, links, links to visualizations and images provided by the other agents and other markdown features to enhance readability and engagement. The blog post has to have at least a paragraph for each section.

    A finalized blog post from the BlogCreatorAgent along with social media posts from SocialCreatorAgent that have all been thoroughly reviewed, and formatted in markdown. The content should be well-structured, with appropriate use of headers, bullet points, links, links to the data visualizations, images, charts and other markdown features to ensure it is both visually appealing and easy to read. The content should be ready for publication and distribution across various channels.
    """),
)

In [ ]:
# The selection function is used to determine which agent should take the next turn in the conversation,
# based on the conversation history and the rules defined in the prompt.

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt="""
    Based on the conversation history, determine the next agent to take a turn. 
    Only return the name of the agent from the following list:
    - LeadResearchAgent
    - DataAnalystAgent
    - ContentCreatorAgent
    - QualityAssuranceAgent

    Rules:
    - LeadResearchAgent starts first.
    - After LeadResearchAgent, DataAnalystAgent speaks.
    - After DataAnalystAgent, ContentCreatorAgent speaks.
    - After ContentCreatorAgent speaks, the QualityAssuranceAgent speaks.
    - If the QualityAssuranceAgent requests changes, restart the cycle with the LeadResearchAgent.
    - If the QualityAssuranceAgent approves the content, the cycle ends.

    History:
    {{$history}}
    """,
)

In [ ]:
# The termination function is used to determine when the conversation should end.
# It checks if the content produced by the agents is complete and valid JSON.

termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt="""
    Determine if the structured content output (article and social_media_posts) are complete.
    If the output is complete and valid, respond with "yes". Otherwise, respond with "no".

    History:
    {{$history}}
    """
)

In [ ]:
# This creates the agent group that will manage the conversation between the agents.

agent_group = AgentGroupChat(
    agents=[lead_research_agent, data_analyst_agent, content_creator_agent, quality_assurance_agent],
    termination_strategy = KernelFunctionTerminationStrategy(
        agents=[quality_assurance_agent],
        function=termination_function,
        kernel=kernel,
        result_parser=lambda result: any("yes" in item.content.lower() for item in result.value) if isinstance(result.value, list) else "yes" in result.value.content.lower(),
        history_variable_name="history",
        maximum_iterations=10
    ),
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=kernel,
        result_parser=lambda result: result.value[0].content.strip() if isinstance(result.value, list) and result.value else result.value.content.strip(),
        agent_variable_name="agents",
        history_variable_name="history"
    )
)

In [ ]:
# This is the question that will be asked to the agents. It should be a clear and concise question that the agents can work on together.
topic = "Inflation in the US and the impact on the stock market in 2025"

initial_message = f"Subject: {topic}\n\nCollaborate to produce structured content output matching the provided schema."
await agent_group.add_chat_message(message=initial_message)

final_response = ""
async for response in agent_group.invoke():
    print(f"# {response.name}:\n{response.content}\n")
    final_response = response.content

In [ ]:
# We run this to extract the JSON from the final response, and display the article and social media posts in a formatted way.

if final_response is None:
    print("No response received from the agents. Run again. :(")
else:
    try:
        # Attempt to extract JSON from the final response
        # and display the article and social media posts in a formatted way.
        markdown_text = json.loads(final_response)
        article = markdown_text.get("article", {})
        posts = markdown_text.get("social_media_posts", [])
        # Display the article in Markdown format
        article_md = f"# {article.get('title')}\n\n{article.get('content')}\n\n**{article.get('call_to_action')}**"
        display(Markdown(article_md))
        # Display the social media posts
        for post in posts:
            print(f"Platform: {post['platform']}")
            print(textwrap.fill(post['content'], width=60))
            print('-' * 60)

    except json.JSONDecodeError as e:
        # It didn't give us JSON, so we print the final response, which is likey in Markdown format.
        display(Markdown(final_response))


